# 평가 (Evaluation)

## 성능평가지표

### 1. 정확도 (Accuracy)

In [ ]:
'''
# 정확도 : 실제 새로운 데이터에서 모델의 예측 데이터가 Label과 얼마나 같은지를 판단하는 지표
                 예측 결과과 동일한 데이터 건수
  정확도 =   -----------------------------------
                전체 예측 데이터 건수

         : 직관적으로 모델의 예측 성능을 나타내는 지표
'''

In [ ]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# 왜곡을 일으키는 분류기
class MyDummyClassfier(BaseEstimator):
  def fit(self, X, y=None):
    pass

  # titanic feature 중에서 Sex Feature가 1이면 0, 0이면 1로 예측하는 모델
  def predict(self, X):
    pred = np.zeros((X.shape[0], 1))
    for i in range(X.shape[0]):
      if X['Sex'].iloc[i] == 1 :
        pred[i] = 0
      else :
        pred[i] = 1
      return pred

In [ ]:
# 결측치 처리 함수
def fillna_nan(df):
  df['Age'].fillna(df['Age'].mean(), inplace=True)
  df['Cabin'].fillna('N', inplace=True)
  df['Embarked'].fillna('N', inplace=True)
  return df

# 레이블 인코딩
def label_encoding_features(df):
  features = ['Sex', 'Embarked']
  for feature in features :
    encoder = LabelEncoder()
    df[feature] = encoder.fit_transform(df[feature])
  return df

# 모델에게 불필요한 특성(feature) 제거
def drop_feature(df):
  df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
  return df

# 최종적으로 전처리를 수행할 함수
def transform_features(df):
  df = fillna_nan(df)
  df = drop_feature(df)
  df = label_encoding_features(df)
  return df

In [ ]:
# 데이터 로딩
titanic_df = pd.read_csv('./titanic_train.csv')

# target (label) 데이터
y_titanic = titanic_df['Survived']

# feature 데이터
X_titanic = titanic_df.drop('Survived', axis=1)

# feature 전처리
X_titanic_df = transform_features(X_titanic)

X_titanic_df.head()

In [ ]:
X_titanic_df.info()

In [ ]:
# 학습 세트, 테스트 세트 분리
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic, test_size=0.2, random_state=11)

In [ ]:
# 왜곡이 생기도록 만든 분류기로 학습/예측/평가
my_clf = MyDummyClassfier()

my_clf.fit(X_train, y_train)

pred = my_clf.predict(X_test)

print(f'왜곡이 생긴 모델의 정확도 : {accuracy_score(y_test, pred)}')

### 2. Confusion Matrix(오차 행렬)

In [ ]:
from IPython.display import Image
Image('./confusion_matrix.png')

In [ ]:
'''
# 오차행렬 (혼동행렬)
 : 학습된 모델이 예측을 수행하면서 얼만큼 헷갈리고 (confusion) 있는지도 함께 보여주는 지표
 : 오류가 얼마인지와 더불어 어떤 유형의 예측 오류가 있는지도 함께 나타내는 지표

                       (TN + TP)
 정확도 = --------------------------------------------
                   (TN + FP + FN + TP)

        =  예측 결과와 실제값이 동일한 수 / 전체 데이터
'''

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, pred)

### 3. Precision(정밀도)와 Recall(재현율)

* 불균형한 데이터세트에서 정확도 보다 더 선호되는 평가지표

* Positive 데이터 세트의 예측 성능에 좀 더 초점을 맞춘 평가지표

In [ ]:
'''
# 정밀도 = TP / (FP + TP)
 : 예측을 Positive 한 대상 중에서 예측과 실제값이 Positive로 일치하는 데이터 비율
 : Positive 예측 성능을 더욱 정밀하게 측정하기 위한 평가 지표 - 양성 예측도

# 재현율 = TP / (FN + TP)
 : 실제값 양성 중 모델이 정확히 양성이라고 예측한 비율. 민감도(Sensitivity), TPR(True Positive Rate)

'''

In [ ]:
from sklearn.metrics import precision_score, recall_score

print(f'정밀도 : {precision_score(y_test, pred)}')
print(f'재현율 : {recall_score(y_test, pred)}')

* 타이타닉 데이터 기반 생존 판단 모델을 이용한 평가 수행

In [ ]:
# 정확도, 정밀도, 재현율, 오차행렬 등을 한꺼번에 구하는 함수
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

def get_clf_eval(y_test, pred):
  confusion = confusion_matrix(y_test, pred)
  accuracy = accuracy_score(y_test, pred)
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)

  print('--- 오차 행렬 -----')
  print(confusion)
  print('-'*40)
  print(f'정확도 : {accuracy}, 정밀도 : {precision}, 재현율 : {recall}')

In [ ]:
# 데이터 로딩
titanic_df = pd.read_csv('./titanic_train.csv')

# target (label) 데이터
y_titanic = titanic_df['Survived']

# feature 데이터
X_titanic = titanic_df.drop('Survived', axis=1)

# feature 전처리
X_titanic_df = transform_features(X_titanic)

X_titanic_df.head()

In [ ]:
# 학습 세트, 테스트 세트 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic, test_size=0.2, random_state=11)

In [ ]:
# 학습 / 예측 / 평가
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(solver='liblinear', random_state=11)

lr_clf.fit(X_train, y_train)

pred = lr_clf.predict(X_test)

get_clf_eval(y_test, pred)

In [ ]:
'''
# 결정 임계값(Threshold)을 조정해서 정밀도 또는 재현율의 수치를 조정 가능하다.
 : 정밀도와 재현율의 관계는 상호 보완적 - 어느 한쪽을 높이면 다른 한쪽은 내려가는 형태
   => 트레이드 오프(trade-off) 관계


# 결정 임계값 : 보통 모델이 판단을 할 때 특정 값을 기준으로 하여 이 기준값 보다 크면 Positive,
                 이 기준값 보다 작으면 Negative 라고 결정을 함.
                 이 판단 기준이 결정임계값임.

                 일반적으로 이진분류에는 0.5가 보통.

'''

In [ ]:
# 현재 모델이 각 데이터에 대해 어떻게 판단하고 있는지 확인 : predict_proba()
pred_proba = lr_clf.predict_proba(X_test)

print('현재 모델의 판단 정보 : \n', pred_proba[:3], '------', pred[:3])

In [ ]:
'''
# 트레이드-오프 시 주의 사항
 - 결정 임계값의 변경은 업무 상황에 맞게 두 정밀도, 재현율 수치를 상호 보완하는 수준에서 적용해야 한다
 - 단순히 하나의 지표를 높이기 위해서 사용하는 것은 안된다!!
'''

### 4. F-1 Score

* 정밀도와 재현율을 결합한 지표

* 정밀도와 재현율이 어느 한쪽으로 치우치지 않은 수치를 나타내면 F1 스코어도 상대적으로 높은 수치를 보임

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_test, pred)

### 5. ROC Curve과 AUC

* 이진 분류의 예측 성능을 평가하는 최근 지표

In [ ]:
'''
# ROC (Receiver Operation Curve) : 수신자 판단 곡선
 - FPR (False Positive Rate) 가 변할 때 TPR(재현율, 민감도) 이 어떻게 변하는 지를 나타내는 곡선
'''

In [ ]:
# 정확도, 정밀도, 재현율, 오차행렬, f1 score, auc_score 등을 한꺼번에 구하는 함수
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred):
  confusion = confusion_matrix(y_test, pred)
  accuracy = accuracy_score(y_test, pred)
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  f1 = f1_score(y_test, pred)
  auc = roc_auc_score(y_test, pred)

  print('--- 오차 행렬 -----')
  print(confusion)
  print('-'*40)
  print(f'정확도 : {accuracy}, 정밀도 : {precision}, 재현율 : {recall}')
  print(f'f1 score : {f1}, AUC : {auc}')

In [ ]:
get_clf_eval(y_test, pred)